In [1]:
import pandas as pd

from relative_path import PATH_DATA, PATH_OUTPUT_DATA, PATH_OUTPUT_GRAPH, PATH_OUTPUT_PROF
from relative_path import TRACK_DATA, ARTIST_DATA, FEATURE_COLS, PROCESSED_FEATURE_DATA

from pandas import DataFrame, Series
from typing import List, Dict

from data_preprocessing import isolate_columns, preprocess_df

## Standard

In [2]:
df_tracks =     pd.read_parquet(TRACK_DATA)
df_artists =    pd.read_parquet(ARTIST_DATA)

# Separate the main criterias from its features.
dfp_tracks, dfp_features = isolate_columns(df_tracks, FEATURE_COLS)

In [3]:
# Preprocess Dataframe - sorting and re-indexing
dfp_tracks = preprocess_df(dfp_tracks, sort_by="popularity")
dfp_artists = preprocess_df(df_artists, sort_by="popularity")

# Rename column
dfp_tracks.rename(columns = {"id":"track_id", "name":"track_name", "popularity": "track_popularity", "artists":"artist_name", "id_artists":"artist_id"}, inplace=True)
dfp_artists.rename(columns = {"id":"artist_id", "name":"artist_name", "popularity": "artist_popularity", "followers":"artist_followers"}, inplace=True)
dfp_features.rename(columns = {"id":"track_id"}, inplace=True)

In [4]:
# Checking and processing the DATATYPES. 
dfp_tracks["dates"] = pd.to_datetime(dfp_tracks["release_date"])
dfp_tracks["release_date"] = pd.to_datetime(dfp_tracks["release_date"])
dfp_tracks["year"]=dfp_tracks["release_date"].dt.year

dfp_tracks.drop(['release_date', 'dates'], axis=1, inplace=True)

In [5]:
# Process Data - Remove popularity values that less than 20. Too much data - distorts the distribution. 
dfp_tracks = dfp_tracks[dfp_tracks["track_popularity"] >= 20]
# dfp_artists = dfp_artists[dfp_artists["artist_popularity"] >= 20]     # This might mess with the apply function - getting the total followers

In [55]:
def remove_characters(string:str, special_char:str = None):
    if special_char == None:
        special_char = "@#$*&"
        
    lst_special_char = [x for x in special_char]
    result = "".join(filter(lambda char: char not in lst_special_char , string))
    return result

def something(lst):
    x = remove_characters(lst, "[]'")
    x = x.split(",")
    x = [a.strip() for a in x]
    
    result = []
    for a in x:
        sample = df_artists[df_artists["artist_id"] == a]
        another = sample["artist_followers"].tolist()
        
        for b in another:
            result.append(int(b))
            
    return sum(result)


In [ ]:
def remove_characters(string:str, special_char:str = None):
    if special_char == None:
        special_char = "@#$*&"
        
    lst_special_char = [x for x in special_char]
    result = "".join(filter(lambda char: char not in lst_special_char , string))
    return result

def apply_fn_followers(string:str) -> int:
    """ Returns the total followers, called under the pandas apply fn. """
    str_process = remove_characters(string, "[]'")
    str_process = str_process.split(",")
    followers_id = [x.strip() for x in str_process]
    
    results = []
    for follower in followers_id:
        match = df_artists[df_artists["artists_id"]==follower]
        match_lst = match["artist_followers"].tolist()
        
        # If no match, then break.
        if match_lst == [] or match_lst == None:
            break
        
        # Flattening the list
        for index, result in enumerate(match_lst): 
            results.append(int(result))
            if index == 1:
                break
            
            
    return sum(result)

## Export - Processed Data

In [ ]:
# Export Final Feature
#dfp_features.to_parquet(PROCESSED_FEATURE_DATA)